# Scraping Barstool Sports' blogger list

#### Python tools and notebook settings

In [8]:
%load_ext lab_black

The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black


In [144]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import altair as alt
import numpy as np
import datetime as dt
import re
import time
import random
from tqdm import tqdm
import urllib.request, json

In [10]:
pd.options.display.max_columns = 100
pd.options.display.max_rows = 1000
pd.options.display.max_colwidth = None

In [11]:
month_year_updated = dt.date.today().strftime("%m_%Y")

---

In [12]:
headers = {
    "sec-ch-ua": '" Not A;Brand";v="99", "Chromium";v="101", "Google Chrome";v="101"',
    "Referer": "https://www.gunviolencearchive.org/",
    "sec-ch-ua-mobile": "?0",
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.0.0 Safari/537.36",
    "sec-ch-ua-platform": '"macOS"',
}

In [13]:
url = "https://www.barstoolsports.com/bloggers"

In [14]:
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text)

In [32]:
cards = soup.find_all("div", class_="jsx-1114159826 tile")

In [66]:
data = []

for card in cards:
    dicts = {
        "author_url": "https://www.barstoolsports.com" + card.find("a")["href"],
        "author_name": card.find("div", class_="authorName").text,
        "author_image": card.find("img", class_="jsx-1114159826")["src"],
    }
    data.append(dicts)

In [67]:
blogger_df = pd.DataFrame(data)
blogger_df.head()

---

## Oh wait, there's an exposed API

#### Get a better list of authors

In [132]:
authors = "https://union.barstoolsports.com/v2/authors"

In [133]:
authors_df = pd.read_json(authors).drop(
    [
        "avatar",
        "id",
        "is_active",
        "merch_url",
        "slug",
        "twitter_handle",
        "short_name",
        "headshot",
    ],
    axis=1,
)

In [135]:
authors_df.head(10)

,name,author_url,bio,twitter,instagram,tiktok,sort,is_suggested
0,El Presidente,/bio/54/el-presidente,"Founder/Chief of Content. Host of The Rundown, Pizza Reviews, The Dave Portnoy Show, BFFs. Founder of DDTG Global. 2x bee sting survivor, US Open champ. I invented the Internet. Brick by brick.",stoolpresidente,stoolpresidente,stoolpresidente,68,True
1,Big Cat,/bio/45/big-cat,"Host of Pardon My Take, Yak, and The Rundown. Football guy’s guy. I only bet overs. Hold record for most stingrays pet at the Shedd Aquarium.",BarstoolBigCat,barstoolbigcat,None,67,True
2,KFC,/bio/44/kfc,"Host of KFC Radio, We Gotta Believe, One Minute Man. Hater of the Year and OG blogger since 2009. Tortured Mets, Jets and Knicks fan.",KFCBarstool,kfcbarstool,kfcbarstool,66,True
3,Kmarko,/bio/47/kmarko,Journalism school dropout. Launched Barstool NY in '09 and Barstool U in '11. Spelling Bee champ. Barstool Idol host. Invented the email newsletter. Will never fix a typo on Barstool Sports.,Kmarkobarstool,kmarkobarstool,None,65,True
4,Feitelberg,/bio/40/feitelberg,I blog and tell my life exploits on KFC Radio. Been to jail for Tom Brady. Mistaken for everyone from ISIS leaders to fish. Creator of the SAFTB movement.,FeitsBarstool,feitelberg,None,64,True
5,Nate,/bio/46/nate,Very good blogger. All world poker player. Head of Barstool DMV. Tambo for O.A.R. Stanley Cup champion.,BarstoolNate,barstoolnate,None,63,True
6,Smitty,/bio/11404/smitty,"Blogger and host of Gametime. Head of Barstool Philly. RnR2 champ. Super Bowl 52 champ. Former pasta pusher, and mush at standup/poker/life.",SmittyBarstool,smittybarstool,smittybarstool1v1,62,False
7,Chaps,/bio/57564/chaps,Host of Zero Blog Thirty and co-host of The Podfathers. Marine and Purple Heart recipient. Never seen a gourd I didn't like. I’ve had diarrhea since Easter 2007.,UncleChaps,unclechaps,None,61,True
8,Liz Gonzales,/bio/63356/liz-gonzales,"Yankee, Packer, and Longhorn fan. Texan. Our resident Mexican. Bring back Blockbuster.",thelizgonzales,thelizgonzales,None,60,True
9,Kayce Smith,/bio/63376/kayce-smith,"Host of the Barstool College Football Show with Pres, Big Cat and Brandon Walker. Host of the Pro Football Football Show. Host of Unnecessary Roughness. Sports blogger. Cancer survivor. Aggie. Clear eyes, full heart, can't lose.",KayceSmith,kayce_smith,None,59,True


---

## Export

In [148]:
authors_df.to_csv("data/processed/barstool_bloggers.csv", index=False)